## Imports

In [57]:
import numpy as np
import pandas as pd

## Load dataset

In [58]:
all_matches = pd.read_csv('dataset/all_matches.csv')

/var/folders/78/_w4q02zn4n3f4dy713ydk9w00000gn/T/ipykernel_90598/592872656.py:1: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  all_matches = pd.read_csv('dataset/all_matches.csv')


## EDA

In [59]:
all_matches.shape

(225954, 22)

In [60]:
all_matches.head()

,match_id,season,start_date,venue,innings,ball,batting_team,bowling_team,striker,non_striker,...,extras,wides,noballs,byes,legbyes,penalty,wicket_type,player_dismissed,other_wicket_type,other_player_dismissed
0,335982,2007/08,2008-04-18,M Chinnaswamy Stadium,1,0.1,Kolkata Knight Riders,Royal Challengers Bangalore,SC Ganguly,BB McCullum,...,1,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
1,335982,2007/08,2008-04-18,M Chinnaswamy Stadium,1,0.2,Kolkata Knight Riders,Royal Challengers Bangalore,BB McCullum,SC Ganguly,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,335982,2007/08,2008-04-18,M Chinnaswamy Stadium,1,0.3,Kolkata Knight Riders,Royal Challengers Bangalore,BB McCullum,SC Ganguly,...,1,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,335982,2007/08,2008-04-18,M Chinnaswamy Stadium,1,0.4,Kolkata Knight Riders,Royal Challengers Bangalore,BB McCullum,SC Ganguly,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,335982,2007/08,2008-04-18,M Chinnaswamy Stadium,1,0.5,Kolkata Knight Riders,Royal Challengers Bangalore,BB McCullum,SC Ganguly,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [61]:
all_matches.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 225954 entries, 0 to 225953
Data columns (total 22 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   match_id                225954 non-null  int64  
 1   season                  225954 non-null  object 
 2   start_date              225954 non-null  object 
 3   venue                   225954 non-null  object 
 4   innings                 225954 non-null  int64  
 5   ball                    225954 non-null  float64
 6   batting_team            225954 non-null  object 
 7   bowling_team            225954 non-null  object 
 8   striker                 225954 non-null  object 
 9   non_striker             225954 non-null  object 
 10  bowler                  225954 non-null  object 
 11  runs_off_bat            225954 non-null  int64  
 12  extras                  225954 non-null  int64  
 13  wides                   7026 non-null    float64
 14  noballs             

### Approximate number of matches

In [62]:
all_matches.shape[0]/240
# since mostly 240 balls are played per match

941.475

### Exact number of matches

In [63]:
len(all_matches['match_id'].unique())

950

## Preprocessing

We are interested in only below features/columns for our prediction:
- batting_team
- bowling_team
- venue
- runs_left
- balls_left
- wickets_left
- total_runs = runs_off_bat + extras
- crr : current run rate
- rrr : required run rate
- result : if the second inning team won ?

### Check if any match has innings >2

In [64]:
all_matches[all_matches['innings']>2]

,match_id,season,start_date,venue,innings,ball,batting_team,bowling_team,striker,non_striker,...,extras,wides,noballs,byes,legbyes,penalty,wicket_type,player_dismissed,other_wicket_type,other_player_dismissed
15417,392190,2009,2009-04-23,Newlands,3,0.1,Kolkata Knight Riders,Rajasthan Royals,CH Gayle,BB McCullum,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15418,392190,2009,2009-04-23,Newlands,3,0.2,Kolkata Knight Riders,Rajasthan Royals,BB McCullum,CH Gayle,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15419,392190,2009,2009-04-23,Newlands,3,0.3,Kolkata Knight Riders,Rajasthan Royals,CH Gayle,BB McCullum,...,1,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15420,392190,2009,2009-04-23,Newlands,3,0.4,Kolkata Knight Riders,Rajasthan Royals,CH Gayle,BB McCullum,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15421,392190,2009,2009-04-23,Newlands,3,0.5,Kolkata Knight Riders,Rajasthan Royals,CH Gayle,BB McCullum,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
198500,1254077,2021,2021-04-25,"MA Chidambaram Stadium, Chepauk, Chennai",4,0.2,Delhi Capitals,Sunrisers Hyderabad,S Dhawan,RR Pant,...,1,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
198501,1254077,2021,2021-04-25,"MA Chidambaram Stadium, Chepauk, Chennai",4,0.3,Delhi Capitals,Sunrisers Hyderabad,RR Pant,S Dhawan,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
198502,1254077,2021,2021-04-25,"MA Chidambaram Stadium, Chepauk, Chennai",4,0.4,Delhi Capitals,Sunrisers Hyderabad,RR Pant,S Dhawan,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
198503,1254077,2021,2021-04-25,"MA Chidambaram Stadium, Chepauk, Chennai",4,0.5,Delhi Capitals,Sunrisers Hyderabad,RR Pant,S Dhawan,...,1,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN


### Remove matches with innings > 2
- as it might affect our model prediction

In [65]:
matches_with_more_innings = all_matches[all_matches['innings']>2].match_id.unique()
matches_with_more_innings

array([ 392190,  419121,  598004,  598017,  729315,  829741, 1082625,
       1175365, 1178426, 1216493, 1216547, 1216512, 1216517, 1254077])

In [66]:
match_df = all_matches[~all_matches['match_id'].isin(matches_with_more_innings)]
match_df

,match_id,season,start_date,venue,innings,ball,batting_team,bowling_team,striker,non_striker,...,extras,wides,noballs,byes,legbyes,penalty,wicket_type,player_dismissed,other_wicket_type,other_player_dismissed
0,335982,2007/08,2008-04-18,M Chinnaswamy Stadium,1,0.1,Kolkata Knight Riders,Royal Challengers Bangalore,SC Ganguly,BB McCullum,...,1,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
1,335982,2007/08,2008-04-18,M Chinnaswamy Stadium,1,0.2,Kolkata Knight Riders,Royal Challengers Bangalore,BB McCullum,SC Ganguly,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,335982,2007/08,2008-04-18,M Chinnaswamy Stadium,1,0.3,Kolkata Knight Riders,Royal Challengers Bangalore,BB McCullum,SC Ganguly,...,1,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,335982,2007/08,2008-04-18,M Chinnaswamy Stadium,1,0.4,Kolkata Knight Riders,Royal Challengers Bangalore,BB McCullum,SC Ganguly,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,335982,2007/08,2008-04-18,M Chinnaswamy Stadium,1,0.5,Kolkata Knight Riders,Royal Challengers Bangalore,BB McCullum,SC Ganguly,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
225949,1312200,2022,2022-05-29,"Narendra Modi Stadium, Ahmedabad",2,17.3,Gujarat Titans,Rajasthan Royals,DA Miller,Shubman Gill,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
225950,1312200,2022,2022-05-29,"Narendra Modi Stadium, Ahmedabad",2,17.4,Gujarat Titans,Rajasthan Royals,Shubman Gill,DA Miller,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
225951,1312200,2022,2022-05-29,"Narendra Modi Stadium, Ahmedabad",2,17.5,Gujarat Titans,Rajasthan Royals,Shubman Gill,DA Miller,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
225952,1312200,2022,2022-05-29,"Narendra Modi Stadium, Ahmedabad",2,17.6,Gujarat Titans,Rajasthan Royals,DA Miller,Shubman Gill,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [67]:
### remove matches with only 1 inning
### there are 2 such rows

## Add a column total_runs
- `total_runs` per ball means `runs_off_bat` + `extras`

In [68]:
match_df['total_runs'] = match_df['runs_off_bat'] + match_df['extras']

/var/folders/78/_w4q02zn4n3f4dy713ydk9w00000gn/T/ipykernel_90598/1691126044.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  match_df['total_runs'] = match_df['runs_off_bat'] + match_df['extras']


In [69]:
match_df.columns

Index(['match_id', 'season', 'start_date', 'venue', 'innings', 'ball',
       'batting_team', 'bowling_team', 'striker', 'non_striker', 'bowler',
       'runs_off_bat', 'extras', 'wides', 'noballs', 'byes', 'legbyes',
       'penalty', 'wicket_type', 'player_dismissed', 'other_wicket_type',
       'other_player_dismissed', 'total_runs'],
      dtype='object')

In [70]:
match_df.groupby(['match_id', 'innings']).sum()['total_runs'].reset_index()

,match_id,innings,total_runs
0,335982,1,222
1,335982,2,82
2,335983,1,240
3,335983,2,207
4,335984,1,129
...,...,...,...
1865,1312198,2,193
1866,1312199,1,157
1867,1312199,2,161
1868,1312200,1,130


In [71]:
total_score_df = match_df.groupby(['match_id', 'innings']).sum()['total_runs'].reset_index()
total_score_df

,match_id,innings,total_runs
0,335982,1,222
1,335982,2,82
2,335983,1,240
3,335983,2,207
4,335984,1,129
...,...,...,...
1865,1312198,2,193
1866,1312199,1,157
1867,1312199,2,161
1868,1312200,1,130


In [72]:
total_score_first_df = total_score_df[total_score_df.innings == 1].rename({'total_runs':'inning1_runs'}, axis=1)
total_score_second_df = total_score_df[total_score_df.innings == 2].rename({'total_runs':'inning2_runs'}, axis=1)
total_score_first_df

,match_id,innings,inning1_runs
0,335982,1,222
2,335983,1,240
4,335984,1,129
6,335985,1,165
8,335986,1,110
...,...,...,...
1860,1304116,1,157
1862,1312197,1,188
1864,1312198,1,207
1866,1312199,1,157


### Add total scores in first inning and second inning

In [73]:
match_df = match_df.merge(total_score_first_df[['match_id', 'inning1_runs']], left_on='match_id', right_on='match_id')
match_df = match_df.merge(total_score_second_df[['match_id', 'inning2_runs']], left_on='match_id', right_on='match_id')
match_df

,match_id,season,start_date,venue,innings,ball,batting_team,bowling_team,striker,non_striker,...,byes,legbyes,penalty,wicket_type,player_dismissed,other_wicket_type,other_player_dismissed,total_runs,inning1_runs,inning2_runs
0,335982,2007/08,2008-04-18,M Chinnaswamy Stadium,1,0.1,Kolkata Knight Riders,Royal Challengers Bangalore,SC Ganguly,BB McCullum,...,NaN,1.0,NaN,NaN,NaN,NaN,NaN,1,222,82
1,335982,2007/08,2008-04-18,M Chinnaswamy Stadium,1,0.2,Kolkata Knight Riders,Royal Challengers Bangalore,BB McCullum,SC Ganguly,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,222,82
2,335982,2007/08,2008-04-18,M Chinnaswamy Stadium,1,0.3,Kolkata Knight Riders,Royal Challengers Bangalore,BB McCullum,SC Ganguly,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,222,82
3,335982,2007/08,2008-04-18,M Chinnaswamy Stadium,1,0.4,Kolkata Knight Riders,Royal Challengers Bangalore,BB McCullum,SC Ganguly,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,222,82
4,335982,2007/08,2008-04-18,M Chinnaswamy Stadium,1,0.5,Kolkata Knight Riders,Royal Challengers Bangalore,BB McCullum,SC Ganguly,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,222,82
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
222124,1312200,2022,2022-05-29,"Narendra Modi Stadium, Ahmedabad",2,17.3,Gujarat Titans,Rajasthan Royals,DA Miller,Shubman Gill,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,130,133
222125,1312200,2022,2022-05-29,"Narendra Modi Stadium, Ahmedabad",2,17.4,Gujarat Titans,Rajasthan Royals,Shubman Gill,DA Miller,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,130,133
222126,1312200,2022,2022-05-29,"Narendra Modi Stadium, Ahmedabad",2,17.5,Gujarat Titans,Rajasthan Royals,Shubman Gill,DA Miller,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,130,133
222127,1312200,2022,2022-05-29,"Narendra Modi Stadium, Ahmedabad",2,17.6,Gujarat Titans,Rajasthan Royals,DA Miller,Shubman Gill,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,130,133


In [74]:
match_df.shape

(222129, 25)

In [75]:
# Add winning team

### Team names

In [76]:
match_df['batting_team'].unique()

array(['Kolkata Knight Riders', 'Royal Challengers Bangalore',
       'Chennai Super Kings', 'Kings XI Punjab', 'Rajasthan Royals',
       'Delhi Daredevils', 'Deccan Chargers', 'Mumbai Indians',
       'Kochi Tuskers Kerala', 'Pune Warriors', 'Sunrisers Hyderabad',
       'Rising Pune Supergiants', 'Gujarat Lions',
       'Rising Pune Supergiant', 'Delhi Capitals', 'Punjab Kings',
       'Lucknow Super Giants', 'Gujarat Titans'], dtype=object)

New teams in 2022 season:
- Lucknow
- Gujrat

Decommissioned Teams:
- Pune
- Kochi

Team names updated:
- Delhi
- Hyderabad
- Gujarat
- Punjab

In [77]:
teams = [
    'Kolkata Knight Riders',
    'Royal Challengers Bangalore',
    'Chennai Super Kings',
    'Rajasthan Royals',
    'Mumbai Indians',
    'Sunrisers Hyderabad',
    'Delhi Capitals',
    'Punjab Kings',
    'Lucknow Super Giants',
    'Gujarat Titans'
]

In [78]:
len(teams)

10

In [79]:
match_df['batting_team'] = match_df['batting_team'].str.replace('Deccan Charges', 'Sunrisers Hyderabad')
match_df['bowling_team'] = match_df['bowling_team'].str.replace('Deccan Charges', 'Sunrisers Hyderabad')

match_df['batting_team'] = match_df['batting_team'].str.replace('Delhi Daredevils', 'Delhi Capitals')
match_df['bowling_team'] = match_df['bowling_team'].str.replace('Delhi Daredevils', 'Delhi Capitals')

match_df['batting_team'] = match_df['batting_team'].str.replace('Kings XI Punjab', 'Punjab Kings')
match_df['bowling_team'] = match_df['bowling_team'].str.replace('Kings XI Punjab', 'Punjab Kings')

match_df['batting_team'] = match_df['batting_team'].str.replace('Gujarat Lions', 'Gujarat Titans')
match_df['bowling_team'] = match_df['bowling_team'].str.replace('Gujarat Lions', 'Gujarat Titans')

In [80]:
match_df = match_df[match_df['batting_team'].isin(teams)]
match_df = match_df[match_df['bowling_team'].isin(teams)]

In [81]:
len(match_df['bowling_team'].unique())

10

In [82]:
match_df.shape

(184445, 25)

### Renaming Venues

In [83]:
match_df['venue'] = np.where(match_df.venue.isin(['Arun Jaitley Stadium',
 'Arun Jaitley Stadium, Delhi', 'Feroz Shah Kotla']), 'Delhi', match_df.venue)

match_df['venue'] = np.where(match_df.venue.isin(['Barabati Stadium']), 'Cuttack', match_df.venue)

match_df['venue'] = np.where(match_df.venue.isin(['Brabourne Stadium',
 'Brabourne Stadium, Mumbai', 'Dr DY Patil Sports Academy',
 'Dr DY Patil Sports Academy, Mumbai', 'Wankhede Stadium',
 'Wankhede Stadium, Mumbai']), 'Mumbai', match_df.venue)

match_df['venue'] = np.where(match_df.venue.isin(['De Beers Diamond Oval']), 'Kimberley', match_df.venue)

match_df['venue'] = np.where(match_df.venue.isin(['Dr. Y.S. Rajasekhara Reddy ACA-VDCA Cricket Stadium']), 'Visakhapatnam', match_df.venue)

match_df['venue'] = np.where(match_df.venue.isin(['Eden Gardens',
 'Eden Gardens, Kolkata']), 'Kolkata', match_df.venue)

match_df['venue'] = np.where(match_df.venue.isin(['Green Park']), 'Kanpur', match_df.venue)

match_df['venue'] = np.where(match_df.venue.isin(['Himachal Pradesh Cricket Association Stadium']), 'Dharamshala', match_df.venue)

match_df['venue'] = np.where(match_df.venue.isin(['Holkar Cricket Stadium']), 'Indore', match_df.venue)

match_df['venue'] = np.where(match_df.venue.isin(['JSCA International Stadium Complex']), 'Ranchi', match_df.venue)

match_df['venue'] = np.where(match_df.venue.isin(['M Chinnaswamy Stadium',
 'M.Chinnaswamy Stadium']), 'Bangalore', match_df.venue)

match_df['venue'] = np.where(match_df.venue.isin(['MA Chidambaram Stadium',
 'MA Chidambaram Stadium, Chepauk',
 'MA Chidambaram Stadium, Chepauk, Chennai']), 'Chennai', match_df.venue)

match_df['venue'] = np.where(match_df.venue.isin(['Maharashtra Cricket Association Stadium',
 'Maharashtra Cricket Association Stadium, Pune', 'Subrata Roy Sahara Stadium']), 'Pune', match_df.venue)

match_df['venue'] = np.where(match_df.venue.isin(['Punjab Cricket Association IS Bindra Stadium',
 'Punjab Cricket Association IS Bindra Stadium, Mohali',
 'Punjab Cricket Association Stadium, Mohali']), 'Mohali', match_df.venue)

match_df['venue'] = np.where(match_df.venue.isin(['Rajiv Gandhi International Stadium',
 'Rajiv Gandhi International Stadium, Uppal',]), 'Hyderabad', match_df.venue)

match_df['venue'] = np.where(match_df.venue.isin(['Narendra Modi Stadium, Ahmedabad', 'Sardar Patel Stadium, Motera']), 'Ahmedabad', match_df.venue)

match_df['venue'] = np.where(match_df.venue.isin(['Saurashtra Cricket Association Stadium']), 'Rajkot', match_df.venue)

match_df['venue'] = np.where(match_df.venue.isin(['Sawai Mansingh Stadium']), 'Jaipur', match_df.venue)

match_df['venue'] = np.where(match_df.venue.isin(['Shaheed Veer Narayan Singh International Stadium']), 'Raipur', match_df.venue)

match_df['venue'] = np.where(match_df.venue.isin(['Zayed Cricket Stadium, Abu Dhabi', 'Sheikh Zayed Stadium']), 'Abu Dhabi', match_df.venue)

In [84]:
match_df['winner'] = np.where((match_df['innings']==1) & (match_df['inning1_runs']>match_df['inning2_runs']), match_df.batting_team, np.nan)
match_df['winner'] = np.where((match_df['innings']==2) & (match_df['inning1_runs']>match_df['inning2_runs']), match_df.bowling_team, match_df.winner)

In [85]:
match_df['winner'] = np.where((match_df['innings']==1) & (match_df['winner'].isna()), match_df.bowling_team, match_df.winner)
match_df['winner'] = np.where((match_df['innings']==2) & (match_df['winner'].isna()), match_df.batting_team, match_df.winner)

In [86]:
second_inn = match_df[match_df['innings']==2].reset_index()
second_inn.shape

(88942, 27)

### Runs left & balls left

In [87]:
second_inn['score']= second_inn.groupby('match_id').cumsum()['total_runs']
second_inn.shape

(88942, 28)

In [88]:
second_inn['runs_left'] = second_inn['inning1_runs'] - second_inn['score'] + 1
second_inn.shape

(88942, 29)

In [89]:
second_inn['balls_left'] = 120 - ((second_inn['ball']*10)%10 + 6*(second_inn['ball']//1)).astype('int')
second_inn.shape

(88942, 30)

### Wickets left

In [90]:
second_inn['wickets_left'] = second_inn['player_dismissed']
second_inn['wickets_left'] = second_inn['wickets_left'].fillna('0').apply(lambda x: x if x=='0' else '1')
second_inn['wickets_left'] = second_inn['wickets_left'].astype('int')
wickets = second_inn.groupby('match_id').cumsum()['wickets_left'].values
second_inn['wickets_left'] = 10 - wickets
second_inn.head()

,index,match_id,season,start_date,venue,innings,ball,batting_team,bowling_team,striker,...,other_wicket_type,other_player_dismissed,total_runs,inning1_runs,inning2_runs,winner,score,runs_left,balls_left,wickets_left
0,124,335982,2007/08,2008-04-18,Bangalore,2,0.1,Royal Challengers Bangalore,Kolkata Knight Riders,R Dravid,...,NaN,NaN,1,222,82,Kolkata Knight Riders,1,222,119,10
1,125,335982,2007/08,2008-04-18,Bangalore,2,0.2,Royal Challengers Bangalore,Kolkata Knight Riders,W Jaffer,...,NaN,NaN,1,222,82,Kolkata Knight Riders,2,221,118,10
2,126,335982,2007/08,2008-04-18,Bangalore,2,0.3,Royal Challengers Bangalore,Kolkata Knight Riders,W Jaffer,...,NaN,NaN,0,222,82,Kolkata Knight Riders,2,221,117,10
3,127,335982,2007/08,2008-04-18,Bangalore,2,0.4,Royal Challengers Bangalore,Kolkata Knight Riders,W Jaffer,...,NaN,NaN,1,222,82,Kolkata Knight Riders,3,220,116,10
4,128,335982,2007/08,2008-04-18,Bangalore,2,0.5,Royal Challengers Bangalore,Kolkata Knight Riders,R Dravid,...,NaN,NaN,1,222,82,Kolkata Knight Riders,4,219,115,10


In [91]:
## consider multiple dismissals also

In [92]:
### crr = runs/overs
second_inn['crr'] = (second_inn['score']*6)/(120 - second_inn['balls_left'])

In [93]:
### rrr = runs_left/balls_left
second_inn['rrr'] = (second_inn['runs_left']*6)/(second_inn['balls_left'])
second_inn.head()

,index,match_id,season,start_date,venue,innings,ball,batting_team,bowling_team,striker,...,total_runs,inning1_runs,inning2_runs,winner,score,runs_left,balls_left,wickets_left,crr,rrr
0,124,335982,2007/08,2008-04-18,Bangalore,2,0.1,Royal Challengers Bangalore,Kolkata Knight Riders,R Dravid,...,1,222,82,Kolkata Knight Riders,1,222,119,10,6.0,11.193277
1,125,335982,2007/08,2008-04-18,Bangalore,2,0.2,Royal Challengers Bangalore,Kolkata Knight Riders,W Jaffer,...,1,222,82,Kolkata Knight Riders,2,221,118,10,6.0,11.237288
2,126,335982,2007/08,2008-04-18,Bangalore,2,0.3,Royal Challengers Bangalore,Kolkata Knight Riders,W Jaffer,...,0,222,82,Kolkata Knight Riders,2,221,117,10,4.0,11.333333
3,127,335982,2007/08,2008-04-18,Bangalore,2,0.4,Royal Challengers Bangalore,Kolkata Knight Riders,W Jaffer,...,1,222,82,Kolkata Knight Riders,3,220,116,10,4.5,11.379310
4,128,335982,2007/08,2008-04-18,Bangalore,2,0.5,Royal Challengers Bangalore,Kolkata Knight Riders,R Dravid,...,1,222,82,Kolkata Knight Riders,4,219,115,10,4.8,11.426087


In [94]:
### won or lost
def result(row):
    return 1 if row['batting_team'] == row['winner'] else 0

In [95]:
second_inn['result'] = second_inn.apply(result, axis=1)
second_inn.head()

,index,match_id,season,start_date,venue,innings,ball,batting_team,bowling_team,striker,...,inning1_runs,inning2_runs,winner,score,runs_left,balls_left,wickets_left,crr,rrr,result
0,124,335982,2007/08,2008-04-18,Bangalore,2,0.1,Royal Challengers Bangalore,Kolkata Knight Riders,R Dravid,...,222,82,Kolkata Knight Riders,1,222,119,10,6.0,11.193277,0
1,125,335982,2007/08,2008-04-18,Bangalore,2,0.2,Royal Challengers Bangalore,Kolkata Knight Riders,W Jaffer,...,222,82,Kolkata Knight Riders,2,221,118,10,6.0,11.237288,0
2,126,335982,2007/08,2008-04-18,Bangalore,2,0.3,Royal Challengers Bangalore,Kolkata Knight Riders,W Jaffer,...,222,82,Kolkata Knight Riders,2,221,117,10,4.0,11.333333,0
3,127,335982,2007/08,2008-04-18,Bangalore,2,0.4,Royal Challengers Bangalore,Kolkata Knight Riders,W Jaffer,...,222,82,Kolkata Knight Riders,3,220,116,10,4.5,11.379310,0
4,128,335982,2007/08,2008-04-18,Bangalore,2,0.5,Royal Challengers Bangalore,Kolkata Knight Riders,R Dravid,...,222,82,Kolkata Knight Riders,4,219,115,10,4.8,11.426087,0


In [96]:
final_df = second_inn[['batting_team', 'bowling_team', 'venue', 'runs_left', 'balls_left', 'wickets_left', 'inning1_runs', 'crr', 'rrr', 'result']]
final_df

,batting_team,bowling_team,venue,runs_left,balls_left,wickets_left,inning1_runs,crr,rrr,result
0,Royal Challengers Bangalore,Kolkata Knight Riders,Bangalore,222,119,10,222,6.000000,11.193277,0
1,Royal Challengers Bangalore,Kolkata Knight Riders,Bangalore,221,118,10,222,6.000000,11.237288,0
2,Royal Challengers Bangalore,Kolkata Knight Riders,Bangalore,221,117,10,222,4.000000,11.333333,0
3,Royal Challengers Bangalore,Kolkata Knight Riders,Bangalore,220,116,10,222,4.500000,11.379310,0
4,Royal Challengers Bangalore,Kolkata Knight Riders,Bangalore,219,115,10,222,4.800000,11.426087,0
...,...,...,...,...,...,...,...,...,...,...
88937,Gujarat Titans,Rajasthan Royals,Ahmedabad,7,15,7,130,7.085714,2.800000,1
88938,Gujarat Titans,Rajasthan Royals,Ahmedabad,7,14,7,130,7.018868,3.000000,1
88939,Gujarat Titans,Rajasthan Royals,Ahmedabad,6,13,7,130,7.009346,2.769231,1
88940,Gujarat Titans,Rajasthan Royals,Ahmedabad,4,12,7,130,7.055556,2.000000,1


In [97]:
final_df = final_df.sample(final_df.shape[0])

### More preprocessing

In [98]:
final_df = final_df[final_df['balls_left'] != 0]

In [99]:
final_df.sample()

,batting_team,bowling_team,venue,runs_left,balls_left,wickets_left,inning1_runs,crr,rrr,result
38429,Sunrisers Hyderabad,Chennai Super Kings,Chennai,193,106,10,209,7.285714,10.924528,0


## Model

In [100]:
X = final_df.iloc[:, :-1]
y = final_df.iloc[:, -1]

In [101]:
from sklearn.model_selection import train_test_split

In [102]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [103]:
X_train

,batting_team,bowling_team,venue,runs_left,balls_left,wickets_left,inning1_runs,crr,rrr
76803,Mumbai Indians,Chennai Super Kings,Dubai International Cricket Stadium,77,50,6,156,6.857143,9.240000
18726,Delhi Capitals,Royal Challengers Bangalore,Bangalore,66,28,5,157,6.000000,14.142857
10656,Mumbai Indians,Kolkata Knight Riders,Mumbai,130,105,10,155,10.400000,7.428571
11028,Mumbai Indians,Chennai Super Kings,Mumbai,147,96,10,180,8.500000,9.187500
32734,Rajasthan Royals,Delhi Capitals,Delhi,67,48,8,152,7.166667,8.375000
...,...,...,...,...,...,...,...,...,...
27363,Punjab Kings,Chennai Super Kings,Chennai,133,71,8,186,6.612245,11.239437
74213,Rajasthan Royals,Delhi Capitals,Mumbai,122,82,7,147,4.105263,8.926829
50728,Delhi Capitals,Mumbai Indians,Mumbai,143,119,10,142,0.000000,7.210084
87538,Lucknow Super Giants,Rajasthan Royals,Mumbai,146,85,7,178,5.657143,10.305882


### Apply one hot encoding on categorial features

In [104]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

col_transform = ColumnTransformer([
    ('col_transform', OneHotEncoder(sparse=False, drop='if_binary'), ['batting_team', 'bowling_team', 'venue'])
], remainder= 'passthrough')

In [105]:
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

In [106]:
pipe = Pipeline(steps=[
    ('transformer', col_transform),
    ('logReg', LogisticRegression(solver='liblinear'))
])

In [107]:
pipe.fit(X_train, y_train)

Pipeline(steps=[('transformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('col_transform',
                                                  OneHotEncoder(drop='if_binary',
                                                                sparse=False),
                                                  ['batting_team',
                                                   'bowling_team',
                                                   'venue'])])),
                ('logReg', LogisticRegression(solver='liblinear'))])

In [108]:
y_pred = pipe.predict(X_test)

In [109]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

0.8140649672907737

### Yet to do:
- Apply Hyperparameter tuning
- try with other algorithms (like randoms forest, XGBoost)
- check random forest for overfitting, and apply regularization
- Show match Proggression with graphs
- unique venues (same venue with multiple names) - **Done**

## Deployment

In [110]:
teams

['Kolkata Knight Riders',
 'Royal Challengers Bangalore',
 'Chennai Super Kings',
 'Rajasthan Royals',
 'Mumbai Indians',
 'Sunrisers Hyderabad',
 'Delhi Capitals',
 'Punjab Kings',
 'Lucknow Super Giants',
 'Gujarat Titans']

In [111]:
final_df['venue'].unique()

array(['Kolkata', 'Hyderabad', 'Chennai', 'Sharjah Cricket Stadium',
       'Ahmedabad', 'Abu Dhabi', 'Mumbai', 'Raipur', 'Pune', 'Jaipur',
       'Bangalore', 'Mohali', 'Newlands',
       'Dubai International Cricket Stadium', 'Rajkot',
       "St George's Park", 'Delhi', 'Dharamshala', 'Cuttack', 'Kingsmead',
       'Visakhapatnam', 'Ranchi', 'Buffalo Park', 'OUTsurance Oval',
       'SuperSport Park', 'New Wanderers Stadium', 'Indore', 'Kanpur',
       'Kimberley'], dtype=object)

In [112]:
import pickle
pickle.dump(pipe, open('pipe.pkl', 'wb'))